In [139]:
import networkx as nx
import ast

ImportError: cannot import name 'from_networkx_graph' from 'sage.graphs.graph_input' (/opt/sagemath-9.3/local/lib/python3.7/site-packages/sage/graphs/graph_input.py)

In [33]:
g = nx.read_gml('testCinfExport.gml')

In [39]:
nx.to_dict_of_dicts(g)

{'[1, 2, 3]': {'[2, 1, 3]': {'weight': '[[1, 0, 0], [0, 0, 0]]'},
  '[1, 3, 2]': {'weight': '[[0, 1, 0], [0, 0, 0]]'},
  '[3, 2, 1]': {'weight': '[[0, 0, 1], [0, 0, 0]]'}},
 '[2, 1, 3]': {'[1, 2, 3]': {'weight': '[[0, 0, 0], [0, 0, 1]]'},
  '[2, 3, 1]': {'weight': '[[0, 0, 1], [0, 0, 0]]'},
  '[3, 1, 2]': {'weight': '[[0, 1, 0], [1, 0, 0]]'}},
 '[2, 3, 1]': {'[3, 2, 1]': {'weight': '[[0, 0, 0], [1, 0, 0]]'},
  '[2, 1, 3]': {'weight': '[[0, 0, 0], [0, 1, 0]]'},
  '[1, 3, 2]': {'weight': '[[0, 0, 0], [0, 0, 1]]'}},
 '[3, 2, 1]': {'[2, 3, 1]': {'weight': '[[0, 1, 0], [0, 0, 0]]'},
  '[3, 1, 2]': {'weight': '[[1, 0, 0], [0, 0, 1]]'},
  '[1, 2, 3]': {'weight': '[[0, 0, 0], [0, 1, 0]]'}},
 '[3, 1, 2]': {'[1, 3, 2]': {'weight': '[[0, 0, 0], [0, 0, 0]]'},
  '[3, 2, 1]': {'weight': '[[0, 0, 0], [0, 0, 0]]'},
  '[2, 1, 3]': {'weight': '[[0, 0, 0], [0, 0, 0]]'}},
 '[1, 3, 2]': {'[3, 1, 2]': {'weight': '[[0, 0, 1], [0, 1, 0]]'},
  '[1, 2, 3]': {'weight': '[[0, 0, 0], [1, 0, 0]]'},
  '[2, 3, 1]': {

In [188]:
def convert_gml(fileName):
    g = nx.read_gml(fileName)
    for edge in g.edges:
        g[edge[0]][edge[1]]['weight'] = ast.literal_eval(g[edge[0]][edge[1]]['weight'])
        g[edge[0]][edge[1]]['weight'] = g[edge[0]][edge[1]]['weight'][0]+g[edge[0]][edge[1]]['weight'][1]
#         print(str(g[edge[0]][edge[1]]['weight']))
#         print(type(g[edge[0]][edge[1]]['weight']))
    return g

def construct_cinf(g, size = -1): #Construct CFKinf complex from graph data - essentially just changing weights to polynomials
    if size == -1:
        size = len(g.get_edge_data(list(g.edges())[0][0],list(g.edges())[0][1])['weight'])  #kind of a mess - just turning the edges
    n = size/2                                                                              #into a list and checking the length of
    preF = LaurentPolynomialRing(GF(2), 'U', n)                                                #the weight of the first edge
    preF.inject_variables()
    Vars = preF.gens()
    F = LaurentPolynomialRing(preF, 'V', n)
    F.inject_variables()
    Vars = Vars + F.gens()
    sageG = DiGraph()
    for edge in g.edges:
        start = edge[0]
        end = edge[1]
        poly = F(1)
        i = 0
        for entry in g[edge[0]][edge[1]]['weight']:
            poly = poly*(Vars[i])**entry
            i = i + 1
#        g[start][end]['weight'] = poly
        sageG.add_edge(start,end,poly)
    return sageG
        
    

In [213]:
g = convert_gml('testCinfExport.gml')
sageG = construct_cinf(g)
# dictg = nx.to_dict_of_dicts(g)
# newg = DiGraph()
# #graphs.graph_input.from_dict_of_dicts(dictg, weighted = True)
# newg
myf = sageG.weighted_adjacency_matrix()

Defining U0, U1, U2
Defining V0, V1, V2


In [214]:
U0*myf

[       0    U0*U1     U0^2        0        0    U0*U2]
[   U0*V0        0        0     U0^2 U0*U2*V1        0]
[   U0*V2        0        0    U0*U2 U0*U1*V0        0]
[       0    U0*V2    U0*V1        0        0    U0*V0]
[       0       U0       U0        0        0       U0]
[   U0*V1        0        0    U0*U1  U0^2*V2        0]

In [212]:
myf

[    0    U1    U0     0     0    U2]
[   V0     0     0    U0 U2*V1     0]
[   V2     0     0    U2 U1*V0     0]
[    0    V2    V1     0     0    V0]
[    0     1     1     0     0     1]
[   V1     0     0    U1 U0*V2     0]

In [205]:
myf.substitute(U0=1)

[    0    U1    U0     0     0    U2]
[   V0     0     0    U0 U2*V1     0]
[   V2     0     0    U2 U1*V0     0]
[    0    V2    V1     0     0    V0]
[    0     1     1     0     0     1]
[   V1     0     0    U1 U0*V2     0]

In [182]:
newg = DiGraph(g, weighted = True)
newg.plot(edge_labels = True)
newg.ed()
#newg.weighted_adjacency_matrix(sparse = True)

AssertionError: 

In [168]:
newg.weighted_adjacency_matrix(base_ring = F)

TypeError: weighted_adjacency_matrix() got an unexpected keyword argument 'base_ring'

In [92]:
g.get_edge_data(list(g.edges())[0][0],list(g.edges())[0][1])['weight'

[1, 0, 0, 0, 0, 0]

In [98]:
preF = LaurentPolynomialRing(GF(2), 4, ('w'))
preF.inject_variables()
F = LaurentPolynomialRing(preF,4,'z')
F.inject_variables()
F.gens()[0]

Defining w0, w1, w2, w3
Defining z0, z1, z2, z3


z0

In [143]:
g

In [115]:
g = convert_gml('testCinfExport.gml')

[1, 0, 0, 0, 0, 0]
<class 'list'>
[0, 1, 0, 0, 0, 0]
<class 'list'>
[0, 0, 1, 0, 0, 0]
<class 'list'>
[0, 0, 0, 0, 0, 1]
<class 'list'>
[0, 0, 1, 0, 0, 0]
<class 'list'>
[0, 1, 0, 1, 0, 0]
<class 'list'>
[0, 0, 0, 1, 0, 0]
<class 'list'>
[0, 0, 0, 0, 1, 0]
<class 'list'>
[0, 0, 0, 0, 0, 1]
<class 'list'>
[0, 1, 0, 0, 0, 0]
<class 'list'>
[1, 0, 0, 0, 0, 1]
<class 'list'>
[0, 0, 0, 0, 1, 0]
<class 'list'>
[0, 0, 0, 0, 0, 0]
<class 'list'>
[0, 0, 0, 0, 0, 0]
<class 'list'>
[0, 0, 0, 0, 0, 0]
<class 'list'>
[0, 0, 1, 0, 1, 0]
<class 'list'>
[0, 0, 0, 1, 0, 0]
<class 'list'>
[1, 0, 0, 0, 0, 0]
<class 'list'>


In [51]:
[1,2]+[3,4]

[1, 2, 3, 4]

In [17]:
conda create -n sage sage python=X

ModuleNotFoundError: No module named 'pyvis'

In [18]:
G = SymmetricGroup([1,2,3,4,5])

In [31]:
G((1,2,3))

(1,2,3)

In [25]:
G.list()[2]

(1,2)

In [26]:
G.list()[3]

(1,4,2,5,3)

In [24]:
G.list()[2]*G.list()[3]

(1,5,3)(2,4)

In [1]:
factor(-2007)

-1 * 3^2 * 223

In [3]:
U, V = var('U V')

In [4]:
U

U

In [5]:
V

V

In [6]:
U*U

U^2

In [165]:
preF = LaurentPolynomialRing(GF(2), 'U', 3)
preF.inject_variables()
F = LaurentPolynomialRing(GF(2), 'V', 3)
F.inject_variables()

Defining U0, U1, U2
Defining V0, V1, V2


In [16]:
F

Multivariate Laurent Polynomial Ring in U, V over Finite Field of size 2